In [1]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

import sys
import os
from tqdm import tqdm
import random

# Add the 'scripts' directory to the Python path
scripts_path = os.path.abspath(os.path.join('..'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)
    
import joblib

# Now you can import the gnn_io module
import gnn_io as gio

import gnn_architectures as garch
# torch.set_printoptions(precision=4, sci_mode=False)

## 1. Define model and parameters

In [2]:
# Define parameters 
project_name = "test_different_parameters"
path_to_save_dataloader = "../../data/data_created_during_training_needed_for_testing/"

loss_fct = torch.nn.MSELoss()
early_stopping_patience = 10

config={
        "epochs": 1000,
        "batch_size": 32,
        "lr": 0.001,
        "loss_fct": "MSELoss",
        "early_stopping_patience": early_stopping_patience,
        "hidden_layer_size": 32,
        "gat_layers": 2,
        "gcn_layers": 0,
        "output_layer": 'gat',
        "in_channels": 6, # dimensions of the x vector + 2 (pos)
        "out_channels": 1,
        # "dropout": 0.15,
    }

unique_model_description = f"mse_loss_hidden_{config['hidden_layer_size']}_gat_{config['gat_layers']}_gcn_{config['gcn_layers']}_lr_{config['lr']}_batch_{config['batch_size']}_epochs_{config['epochs']}_early_{config['early_stopping_patience']}_out_{config['output_layer']}_"

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
        },
    # 'batch_size': {
    #     # integers between 32 and 256
    #     # with evenly-distributed logarithms 
    #     'distribution': 'q_log_uniform_values',
    #     'q': 8,
    #     'min': 32,
    #     'max': 256,
    #   }
    
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'hidden_layer_size': {
            'values': [16, 32, 64]
        }
    }
}


# # Fix some parameters over which we do not want to sweep
# parameters_dict.update({
#     'epochs': {
#         'value': 1}
#     })

# # Add distributions for some parameters, not just randomly selecting any of the values
# parameters_dict.update({
    
#     })

import pprint
pprint.pprint(sweep_config)

{'hidden_layer_size': {'values': [16, 32, 64]},
 'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'optimizer': {'values': ['adam', 'sgd']},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0}}}


## 2. Load data

In [4]:
# Reconstruct the Data objects
data_dict_list = torch.load('../../data/train_data/dataset_1pm_0-3100.pt')
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y']) for d in data_dict_list]
dataset = gio.normalize_dataset(datalist, y_scalar=None, x_scalar_list=None, pos_scalar=None, directory_path=path_to_save_dataloader)

# baseline_error = gio.compute_baseline_of_no_policies(dataset=dataset, loss_fct=loss_fct)
# print(f'Baseline error no policies: {baseline_error}')

# baseline_error = gio.compute_baseline_of_mean_target(dataset=dataset, loss_fct=loss_fct)
# print(f'Baseline error mean: {baseline_error}')

In [6]:
wandb.login()

early_stopping = gio.EarlyStopping(patience=early_stopping_patience, verbose=True)
# gnn_instance = garch.MyGnn(in_channels=config['in_channels'], out_channels=config['out_channels'], hidden_size=config['hidden_layer_size'], gat_layers=config['gat_layers'], gcn_layers=config['gcn_layers'], output_layer=config['output_layer'])
# model = gnn_instance.to(device)

sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

def train():
    wandb.init(
    project=project_name,
    config = config)
    
    config = wandb.config
    
    gnn_instance = garch.MyGnn(in_channels=config['in_channels'], out_channels=config['out_channels'], hidden_size=config['hidden_layer_size'], gat_layers=config['gat_layers'], gcn_layers=config['gcn_layers'], output_layer=config['output_layer'])
    model = gnn_instance.to(device)     
    wandb.watch(model)

    train_dl, valid_dl, test_dl = gio.create_dataloaders(batch_size = config['batch_size'], dataset=dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
    # gio.save_dataloader(test_dl, path_to_save_dataloader + 'test_dl_' + unique_model_description + '.pt')
    # gio.save_dataloader_params(test_dl, path_to_save_dataloader + 'test_loader_params_' + unique_model_description+ '.json')
    garch.train(model, config=config, 
                                loss_fct=loss_fct, 
                                optimizer=torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=0.0),
                                train_dl=train_dl, valid_dl=valid_dl,
                                device=device, early_stopping=early_stopping)
        

wandb.agent(sweep_id=sweep_id, function=train, count=5)

# best_val_loss, best_epoch = garch.train(model, config=config, 
#                                 loss_fct=loss_fct, 
#                                 optimizer=torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=0.0),
#                                 train_dl=train_dl, valid_dl=valid_dl,
#                                 device=device, early_stopping=early_stopping)

# model.to_onnx(path_to_save_dataloader + 'model_' + unique_model_description + '.onnx', test_dl, device)
# wandb.save(path_to_save_dataloader + 'model_' + unique_model_description + '.onnx')

400 response executing GraphQL.
{"errors":[{"message":"Invalid sweep config: Sweep config must contain parameters section","path":["upsertSweep"]}],"data":{"upsertSweep":null}}
wandb: ERROR Error while calling W&B API: Invalid sweep config: Sweep config must contain parameters section (<Response [400]>)


UsageError: Invalid sweep config: Sweep config must contain parameters section